## This notebook is part of Hadoop and Spark training delivered by CERN IT


### SPARK DataFrame Hands-On Lab
Contact: Luca.Canali@cern.ch

### Objective: Perform Basic DataFrame Operations
1. Creating DataFrames
2. Select columns
3. Add, rename and drop columns
4. Filtering rows
5. Aggregations

## Exercises and solutions

Reminder: documentation at 
https://spark.apache.org/docs/latest/api/python/index.html

In [ ]:
# Create the SparkSession
# and read the dataset

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .master("local[*]") \
        .appName("DataFrame HandsOn 1") \
        .config("spark.ui.showConsoleProgress","false") \
        .getOrCreate()

online_retail_schema="InvoiceNo int, StockCode string, Description string, Quantity int,\
InvoiceDate timestamp,UnitPrice float,CustomerId int, Country string"

df = spark.read \
        .option("header", "true") \
        .option("timestampFormat", "M/d/yyyy H:m")\
        .csv("../data/online-retail-dataset.csv.gz",
             schema=online_retail_schema)

Task: Show 5 lines of the "description" column 

In [2]:
df.select("description").show(5,truncate=False)

+-----------------------------------+
|description                        |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+
only showing top 5 rows



Task: Count the number of distinct invoices in the dataframe

In [3]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("InvoiceNo")).show()

+-------------------------+
|count(DISTINCT InvoiceNo)|
+-------------------------+
|                    22061|
+-------------------------+



Task: Find out in which month most invoices have been issued

In [5]:
from pyspark.sql.functions import month
df.groupby(month("InvoiceDate")).count().sort('count').show()

+------------------+-----+
|month(InvoiceDate)|count|
+------------------+-----+
|                 2|27707|
|                 4|29916|
|                 1|35147|
|                 8|35284|
|                 3|36748|
|                 6|36874|
|                 5|37030|
|                 7|39518|
|                 9|50226|
|                10|60742|
|                12|68006|
|                11|84711|
+------------------+-----+



Task: Filter the lines where the Quantity is more than 30

In [6]:
df.where("Quantity > 30").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerId|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536367|    84879|ASSORTED COLOUR B...|      32|2010-12-01 08:34:00|     1.69|     13047|United Kingdom|
|   536370|    10002|INFLATABLE POLITI...|      48|2010-12-01 08:45:00|     0.85|     12583|        France|
|   536370|    22492|MINI PAINT SET VI...|      36|2010-12-01 08:45:00|     0.65|     12583|        France|
|   536371|    22086|PAPER CHAIN KIT 5...|      80|2010-12-01 09:00:00|     2.55|     13748|United Kingdom|
|   536374|    21258|VICTORIAN SEWING ...|      32|2010-12-01 09:09:00|    10.95|     15100|United Kingdom|
|   536376|    22114|HOT WATER BOTTLE ...|      48|2010-12-01 09:32:00|     3.45|     15291|United Kingdom|
|   536376|    21733|RED HAN

Task: Show the four most sold items (by quantity)

In [7]:
from pyspark.sql.functions import desc, asc, expr
(df.groupBy("Description")
     .agg(expr("sum(Quantity) as totalQuantity"))
     .sort("totalQuantity", ascending=False)
     .show(4))


+--------------------+-------------+
|         Description|totalQuantity|
+--------------------+-------------+
|WORLD WAR 2 GLIDE...|        53847|
|JUMBO BAG RED RET...|        47363|
|ASSORTED COLOUR B...|        36381|
|      POPCORN HOLDER|        36334|
+--------------------+-------------+
only showing top 4 rows



Bonus question: why do these two operations return different results? Hint: look at the documentation 

In [8]:
print(df.select("InvoiceNo").distinct().count())
from pyspark.sql.functions import countDistinct
df.select(countDistinct("InvoiceNo")).show()

22062
+-------------------------+
|count(DISTINCT InvoiceNo)|
+-------------------------+
|                    22061|
+-------------------------+



As you can see from the output of `countDistinct`, internally it runs `count(DISTINCT`, which excludes `null`s.

https://spark.apache.org/docs/latest/api/sql/#count

* `count()` Returns the total number of retrieved rows, including rows containing null

* `count(DISTINCT expr[, expr...])` - Returns the number of rows for which the supplied expression(s) are unique and non-null.